# The Euler-Bernoulli beam model

Model: 2D (planar) beam reduced to 1D problem. We study the deformation of the midplane under the assumptions that after the deformation the normals to the midplane:

* do not bend
* do not stretch
* remain orthogonal to the midplane.

This theory is intended for thin beams under small strains even with large global deformations: it is a physically linear but geometrically non-linear theory.

For thicker beams, Timoshenko's theory, which accounts for internal shear forces, yields more accurate predictions.

Fix $\omega = (a, b)$ to be the midplane of the beam $\Omega = \omega \times
(- h / 2, h / 2) \subset \mathbb{R}^2$.

## Derivation

Ad-hoc stuff … compute … hack … compute … and:

$$\frac{\mathrm{d}^2}{\mathrm{d} x^2} \left( b(x) \frac{\mathrm{d}^2}{\mathrm{d} x^2}u(x) \right) = f$$

where $b(x) = E(x) I(x)$ is the product of Young's modulus $E$ and the area moment of inertia of the beam $I$. For a steel beam (with < 0.3% carbon) at 21°C, $E = 203.4 \cdot 10^9 \text{Pa}$ and if it has a constant square cross section of side 0.01m, then $I=8 \cdot 10^{-10}$.


## Weak formulation

Write $\nabla u = \frac{\mathrm{d}}{\mathrm{d} x} u$ and $\Delta u =
\frac{\mathrm{d}^2}{\mathrm{d} x^2} u$. Let $V$ be a subspace of $H^2
(\Omega)$ to be specified later. Multiplying the equation by a
test function $v \in V$ and integrating by parts we arrive at

$$ \int_{\Omega} b \Delta u \Delta v \mathrm{d} x - [b \Delta u \nabla v]_a^b
   + [\nabla (b \Delta u) v]_a^b = \int_{\Omega} fv \mathrm{d} x. $$

In order for these integrals to make sense we may take $b \in L^{\infty}
(\Omega)$ and $f \in L^2 (\Omega)$ (actually even $H^{- 2} (\Omega)$). The
definition of $V$ and the final form of the equation are determined by our
choices for the four boundary conditions that have to be specified:

**Essential boundary conditions:** We fix either the
**deflections** $u (\alpha)$ or the **slopes** $u' (\alpha)$
or both at the ends of the beam $\alpha \in \{a, b\}$. These conditions are
incorporated into the definition of $V$. For example if we **clamp**
the beam at an horizontal position we have

$$ V = V_{\text{clamped}} = \{ v \in H^2 (\Omega) : v (\alpha) = v_{\alpha},
   v' (\alpha) = v'_{\alpha}, \alpha = 1, 2 \} . $$

**Natural boundary conditions:** For $\alpha \in \{a, b\}$, we can fix
the **bending moment**:

$$ M (\alpha) = (b \Delta u) (\alpha), $$

which for general $x \in \Omega$ is the torque exerted by forces surrounding
$x$. If, for example we find solutions such that $M (a) = 0$, then we are
assuming that the left end of the beam is free to rotate, i.e. that it
undergoes no bending due to torque. Alternatively we can set the
**shear force** at $\alpha$:

$$ F (\alpha) = [\nabla (b \Delta u)] (\alpha), $$

which is the resultant of transversal forces at $x \in \Omega$. This will in
general be zero at the ends.

If we set $\Gamma_M, \Gamma_F \subset \{ a, b \}$, after choosing some
combination of the conditions the problem is: Find $u \in V$ such that for all
$v \in V$:

$$ \int_{\Omega} b \Delta u \Delta v \mathrm{d} x = \int_{\Omega} fv
   \mathrm{d} x + \int_{\Gamma_M} M \nabla v \mathrm{d} s - \int_{\Gamma_F} Fv
   \mathrm{d} s $$
   
$V, \Gamma_M, \Gamma_F$ to be (consistently) specified.

## Discretization

Recall that $H^2(a,b) \in C^1(a,b)$ by the Sobolev embeddings.

Even though cuadratic polynomials might be enough, we want to construct a Ciarlet finite element, i.e. need unisolvent set of degrees of freedom, which requires at least cubic polynomials ... [elaborate, see p.218 of ...]

Use Cubic Hermite elements => $H^2$ conforming in $\mathbb{R}$.

# Common code

Because we will be doing several examples it is convenient to factor out most of the stuff. The resulting code is somewhat ugly, with a nasty mixture of global state and unintuitive parameters, but elegance is not the point here.

In [ ]:
from dolfin import *
import nbimporter
from boundary import *
import autograd as ad
import autograd.numpy as np
%matplotlib inline
import matplotlib.pyplot as pl

# Alias
NeumannBC = DirichletBC

# OLD Elasticity parameters
#E = 1e9
#nu = 0.3
#mu = E/(2.0*(1.0 + nu))
#lmbda = E*nu/((1.0 + nu)*(1.0 - 2.0*nu))

# Beam and problem parameters. Assume an homogenous steel beam of constant cross-section
E = 203.4e9  # Steel (< 0.3% carbon) @ 21°C ~ 203.38 GPa
I = 8e-10    # Second moment of inertia / area (in m^4, for square beam of side 0.01m)
g = 9.8      # Gravity m/s^2

# Basic setup
LEFT = 0.0
RIGHT = 2.0
mesh = IntervalMesh(10, LEFT, RIGHT)
V = FunctionSpace(mesh, 'Hermite', 3)
b = Constant(E*I) # HACK
# We will use this to mark boundaries for boundary conditions
exterior_facet_domains = FacetFunction("uint", mesh)
#exterior_facet_domains.set_all(1)

class Right(SubDomain):
    """ Right end of the beam. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return on_boundary and np.isclose(x[0], RIGHT)

class Left(SubDomain):
    """ Left end of the beam. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return on_boundary and np.isclose(x[0], LEFT)

class Boundary(SubDomain):
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return on_boundary and (near(x[0], LEFT) or near(x[0], RIGHT))
    

def solve_with_bcs(essential_bcs, natural_bcs=None, external_force=None):
    """ Sets up and solves the beam problem for the given boundary
    conditions.
    
    Arguments:
    ----------
        essential_bcs: a tuple of [DirichletBC, NeumannBC] (either can
                       be None but not both)
        natural_bcs: a tuple of ufl Forms (b,b') representing:
                     b: additional terms for the bilinear form a()
                     b': additional terms for the right hand side
        external_force: gravity is used if set to None.

    Returns:
    --------
        A Function in the space where the essential bcs are defined.
    """

    if essential_bcs[0] is not None:
        V = essential_bcs[0].function_space()
    elif essential_bcs[1] is not None:
        V = essential_bcs[1].function_space()
    else:
        raise Exception("I need at least one essential BC.")

    # n point Gauss quadrature is exact for polynomials of order 2n-1:
    fc_params = {'representation': 'quadrature', 'quadrature_degree': 2}
    
    if natural_bcs is None:
        natural_bcs = [None, None]
    u = TrialFunction(V)
    v = TestFunction(V)

    f = Constant(-g) if external_force is None else external_force
    a = b * u.dx(0).dx(0) * v.dx(0).dx(0)*dx
    if natural_bcs[0] is not None:
        a = a + natural_bcs[0]
    
    L = f*v*dx
    Fp = assemble(L, form_compiler_parameters=fc_params)
    if natural_bcs[1] is not None:
        L = L + natural_bcs[1]
    u = Function(V)

    A = assemble(a, form_compiler_parameters=fc_params)
    F = assemble(L, form_compiler_parameters=fc_params)
    apply_dirichlet_hermite(A, F, essential_bcs[0])
    apply_neumann_hermite(A, F, essential_bcs[1])

    u = Function(V)
    U = u.vector()
    solve(A, U, F)
    
    return u, A, F, Fp

In [ ]:
def plot_beam(solution, beam_width=0.01, magnification=1.0, title=None, label=None):
    """ Draws the deformed beam and its midplane. 
    
    Arguments
    ---------
        solution: a dolfin's Function giving the vertical displacement
                  of the midplane of the beam
        beam_width: the upper and lower layers of the beam will be drawn
                    at half this value away from the midplane
    """
    
    xx = mesh.coordinates().flatten()
    u = solution[0].compute_vertex_values(mesh) * magnification
    offset = beam_width / 2
    pl.figure(figsize=(10,3))
    pl.plot(xx, u, c='brown', label=label if label else '$u_h$')
    pl.plot(xx, u-offset, c='brown', lw=0.5)
    pl.plot(xx, u+offset, c='brown', lw=0.5)
    pl.ylabel("%s deflection" % ("%.1f *" % magnification if magnification != 1. else ""))
    pl.xlabel("x")
    pl.xlim(xx.min(), xx.max())
    pl.ylim(-beam_width+(np.min(u)-offset)*2, beam_width*5)
    pl.title(title if title is not None else "")

# Plug-in test

To test the code we assume a deflection given by $u(x)=-\frac{x^4}{16}$ and constant $b(x) = 1$ over the interval $(a,b) = (0,1)$. Substituting into the equation this yields a right hand side $f(x) = - \frac{3}{2}$.

The essential boundary conditions are included into the solution space and given by $u(a)$ and $u'(a)$. At the opposite side of the beam we have the natural ones $M_b = u''(b)$ and  $F_b = u'''(b)$.

In [ ]:
xx = V.mesh().coordinates().flatten()
u_exact = lambda x: -1./16 * x**4
up_exact = ad.elementwise_grad(u_exact, 0) #lambda x: -1./4 * x**3
upp_exact = ad.elementwise_grad(up_exact, 0) #lambda x: -3./4 * x**2
uppp_exact = ad.elementwise_grad(upp_exact, 0) #lambda x: -3./2 * x
pl.figure(figsize=(6,3))
pl.plot(xx, u_exact(xx), label='$u_e$')
pl.plot(xx, up_exact(xx), label='$u_e\'$')
pl.plot(xx, upp_exact(xx), label='$u_e\'\'$')
pl.plot(xx, uppp_exact(xx), label='$u_e\'\'\'$')
pl.xlim((LEFT, RIGHT))
pl.legend(loc='lower left')
_ = pl.title("Analytic solution and its derivatives")

## FAIL:

WHAT'S GOING ON!?!?!?!?!?!?!?!

<strike>The magnitude of the solution depends on how fine the grid is, hinting at some problem with the quadratures (because the definition of the basis functions in the global disrete space involves the jacobian of the geometric transformation from the reference simplex to each simplex in the physical domain).</strike>

To test this idea, change the grid size in `IntervalMesh()` to 10, 100, 1000 and verify that the solution is off by a factor of <strike>1e8, 1e16 and 1e24 respectively. Correcting by these numbers, the error falls down to numbers of order 1e-11, 1e-14 and 1e9 respectively.</strike>

In [ ]:
#pl.figure(figsize=(6, 6*2.3))
# n point Gauss quadrature is exact for polynomials of order 2n-1:
fc_params = {'representation': 'quadrature', 'quadrature_degree': 2}

errors = {1: [], 2: [], 4:[], np.inf: []}
at_point = {0.2: [], 0.4: [], 0.6: [], 0.8: []}
#mesh_sizes = np.arange(10, 1011, 200)
mesh_sizes = np.arange(1, 11, 3)

for num_cells in mesh_sizes:
    mesh = IntervalMesh(num_cells, LEFT, RIGHT)
    V = FunctionSpace(mesh, "Hermite", 3)
    exterior_facet_domains = FacetFunction("uint", mesh)

    essential_boundary = Left()
    essential_boundary.mark(exterior_facet_domains, 1)
    natural_boundary = Right()
    natural_boundary.mark(exterior_facet_domains, 2)
    ds = ds(subdomain_data = exterior_facet_domains)

    left_position   = project(Constant(u_exact(LEFT)), V)
    left_derivative = project(Constant(up_exact(LEFT)), V)
    right_moment    = project(Constant(upp_exact(RIGHT)), V)
    right_shear     = project(Constant(uppp_exact(RIGHT)), V)
    external_force  = project(Constant(-3./2), V)

    v = TestFunction(V)
    u, A, F, Fp = solve_with_bcs(
                    essential_bcs = [DirichletBC(V, left_position, exterior_facet_domains, 1),
                                     NeumannBC(V, left_derivative, exterior_facet_domains, 1)],
                    natural_bcs   = [None, right_moment * v.dx(0) * ds(2) - 
                                           right_shear * v * ds(2)],
                    external_force = external_force)
    
    #print("Fp=%s" % Fp.array().round(2))
    #print("F =%s" % F.array().round(2))

    xx = V.mesh().coordinates().flatten()
    uu = u_exact(xx)
    uv = u.compute_vertex_values(mesh) # np.array([u(x) for x in xx])
    diff = uu - uv
    pl.plot(xx, uv, '--', label='$u_{%d}$' % num_cells)
    for k in errors.keys():
        errors[k].append(np.linalg.norm(diff, ord=k)/
                         (num_cells**k if k is not np.inf else num_cells))
#                         np.linalg.norm(uu, ord=k))
    for k in at_point.keys():
        at_point[k].append(u_exact(k) - u(k))

pl.plot(xx, uu, '-', label="$u_e$", color='black')
pl.ylim((-1, 0.1))
_ = pl.legend(loc='lower left')

**FIXME: We are multiplying the error by $h^k$, not truly normalizing by $||u_e||$.**

In [ ]:
pl.figure(figsize=(10,4))
pl.subplot(1,2,1)
for norm, vals in errors.iteritems():
    pl.plot(mesh_sizes, vals, label="%s-norm" % norm)
pl.legend(fontsize='small')
pl.xlabel("Mesh size")
_ = pl.title("'Normalized' error")
pl.subplot(1,2,2)
for pt, vals in at_point.iteritems():
    pl.plot(mesh_sizes, vals, label="$x=%.1f$" % pt)
pl.legend(fontsize='small')
pl.xlabel("Mesh size")
_ = pl.title("Distance at points")

Compute

$$ \int_{\Gamma_2} b v' \mathrm{d}s, $$

where $\Gamma_2 \subset \partial \Omega$ is the part of the boundary where the bending moment of the solution is set to $b$ (the rightmost point).

In [ ]:
G = assemble(boundary_moment*v.dx(0)*ds(2))
coeffs = np.nonzero(G.array() != 0)[0]
coords = V.tabulate_dof_coordinates()[coeffs]
print("Coefficients %s are non-zero and have coordinates %s" % (coeffs, coords))
print("A[coeffs] =\n%s" % A.array()[coeffs])

In [ ]:
r = Function(V, G)

plot(r)
pl.plot(xx, [r(x) for x in xx])
_ = pl.ylim((-1,2))

In [ ]:
t = Function(V, Fp)

xx = np.linspace(0,2,20)
plot(t)
pl.plot(xx, [t(x) for x in xx])
pl.ylim((-11,-9))

In [ ]:
for k, e in errors.iteritems():
    pl.plot(mesh_sizes, e, label="$l_{%s}$" % k, marker='.')
pl.title("Error as a function of the number of cells")
_ = pl.legend(loc='upper left')

In [ ]:
for k, val in at_point.iteritems():
    pl.plot(mesh_sizes, val, label="$d_{%.1f}$" % k, marker='.')
pl.title("Difference at points")
_ = pl.legend(loc='upper left')

In [ ]:
plot_beam(u, label="$u_h$")
pl.plot(xx, uu, label="$u_e$")
pl.ylim((-1.1,0.1))
pl.legend(fancybox=True, loc='lower left')
_ = pl.title("Fail!")

Errors:
* mesh_size = 10, error_linf: 0.
* mesh_size = 100, error_linf: 0.166
* 

# FIXME:

**Are the natural bcs for 3rd and 4th derivatives being enforced if they are constant functions? It seems like the computed solution yields something at most quadratic at the boundary, i.e. with those derivatives set to 0** 

# ALSO: are these integrals right?

Note how multiplying by $v'$ introduces a factor equal to the number of cells in the domain.

In [ ]:
mesh = IntervalMesh(10, LEFT, RIGHT)
V = FunctionSpace(mesh, "Hermite", 3)
exterior_facet_domains = FacetFunction("uint", mesh)

essential_boundary = Left()
essential_boundary.mark(exterior_facet_domains, 1)
natural_boundary = Right()
natural_boundary.mark(exterior_facet_domains, 2)
ds = ds(subdomain_data = exterior_facet_domains)

left_position   = project(Constant(u_exact(LEFT)), V)
left_derivative = project(Constant(up_exact(LEFT)), V)
boundary_moment = project(Constant(upp_exact(RIGHT)), V)
boundary_shear  = project(Constant(uppp_exact(RIGHT)), V)
external_force  = project(Constant(-3./2), V)

v = TestFunction(V)
u = solve_with_bcs(essential_bcs = [DirichletBC(V, left_position, exterior_facet_domains, 1),
                                    NeumannBC(V, left_derivative, exterior_facet_domains, 1)],
                   natural_bcs   = [None, inner(boundary_moment, v.dx(0))*ds(2) - 
                                          inner(boundary_shear, v)*ds(2)],
                   external_force = external_force)

In [ ]:
boundary_moment = project(Constant(-0.75), V)
print(assemble(boundary_moment*ds(2)))
print(assemble(boundary_moment*v.dx(0)*ds(2)).array())

In [ ]:
boundary_shear  = project(Constant(-1.5), V)
print(assemble(boundary_shear*ds(2)))
print(assemble(boundary_shear*v*ds(2)).array())

In [ ]:
V.tabulate_dof_coordinates()

In [ ]:
ds.subdomain_data().array()

# Clamped beam

We look for a solution with clamped boundaries, i.e. $u(a)=u(b)=u'(a)=u'(b)=0$. Because of the essential constraints, the terms corresponding to the natural ones in the weak formulation vanish and we are left with:

$$ V = V_{\text{clamped}} = \{ v \in H^2 (\Omega) : v (\alpha) = v_{\alpha},
   v' (\alpha) = v'_{\alpha}, \alpha = 1, 2 \} . $$
   
and $\Gamma_M = \Gamma_F = \emptyset$.

In [ ]:
zero_constant = project(Constant(0.), V)
essential_bcs = [DirichletBC(V, zero_constant, Boundary()),
                 NeumannBC(V, zero_constant, Boundary())]
u = solve_with_bcs(essential_bcs)

plot_beam(u, magnification=2)

# Cantilevered beam

One boundary is clamped, e.g. $u(a)=u'(a)=0$, and the other is left hanging freely, so that no bending moment (i.e. no torque) and no shear force appear, e.g. assuming constant $E$ and $I$: $u''(b)=u'''(b)=0$.

We can generalise this to the case where we hang a mass from the free end of the beam. The downward force will introduce shear, e.g. $u'''(b) = - m g$, with $m$ the mass of the object and $g$ gravity. Just set $m=0$ for the first situation.

In [ ]:
zero_constant = project(Constant(0.), V)

# Kg of mass hanging from the right end, set to zero for a free end
masses = np.linspace(10,1010, 5)
for i, m in enumerate(masses):
    #m = 100

    # Cantilever BCs: 0 bending and shear = -m*g at the right side
    essential_boundary = Left()
    essential_boundary.mark(exterior_facet_domains, 1)
    natural_boundary = Right()
    natural_boundary.mark(exterior_facet_domains, 2)
    ds = ds(subdomain_data=exterior_facet_domains)

    boundary_moment = Constant(0)
    boundary_shear = Constant(m*g/(E*I))

    v = TestFunction(V)
    u = solve_with_bcs(essential_bcs = [DirichletBC(V, zero_constant, exterior_facet_domains, 1),
                                        NeumannBC(V, zero_constant, exterior_facet_domains, 1)],
                       natural_bcs   = [None, boundary_moment * v.dx(0) * ds(2) -
                                              boundary_shear * v * ds(2)])
    xx = mesh.coordinates().flatten()
    uu = u[0].compute_vertex_values(mesh)
    pl.plot(xx, uu, label='$u_{%d}$' % m)
    #plot_beam(u, title="Cantilevered beam with weight hanging")
_ = pl.legend(loc="lower left")

# Simply supported beam

The beam rests on two point supports at its ends. Displacements are fixed at both ends with $u(a)=u(b)=0$, but since the beam is free to rotate, it experiences no torque at these points, i.e. the bending moments are zero: $u''(a)=u''(b)=0$, so we can plug in the natural boundary condition $M=0$ and the problem is: Find $u \in V = H^2_0(\omega)$ such that for all $v \in V$:

$$ \int_{\omega} b \Delta u \Delta v \mathrm{d} x + \int_{\gamma} \nabla (b \Delta u) v 
\mathrm{d} s = \int_{\omega} fv \mathrm{d} x. $$

This means we need to change the bilinear form $a$ in the problem setting, and in our hackish setup we do this by setting the first element of `natural_bcs` in the call to `solve_with_bcs`.

In [ ]:
# FIXME? Is it ok to define a form with new variables u,v, 
# then add it to a() in solve_with_bcs?
u = TrialFunction(V)
v = TestFunction(V)
u = solve_with_bcs(essential_bcs = [DirichletBC(V, zero_constant, Boundary()), None],
                   natural_bcs   = [inner((b*u.dx(0).dx(0)).dx(0), v)*ds, None]) 
plot_beam(u)

### Debugging history

** On constants and Constants **

Playing with the declarations of the constants in the problem setting yields confusing results. For instance, declaring `boundary_moment = Constant(-0.75)*Constant(E)*Constant(I)` is not equivalent to `boundary_moment = Constant(-0.75*E*I)` and, as a matter of fact, completely screws the solution inverting the slope and producing an increasing function.

** Boundary conditions **

DirichletBC thinks that all dofs at a node are point evaluations so it resets many rows of the mass matrix which it should not. There's a manual hack-around in `apply_dirichlet_hermite()`, but ideally I need to fix either DirichletBC or the dofmap returned by the element or something like that.

# Questions

* What is the difference between `Cell.get_coordinate_dofs()` ("Get cell coordinate dofs (not vertex coordinates)") and `Cell.get_vertex_coordinates()` ("Get cell vertex coordinates (not coordinate dofs)"). How can a Cell in a mesh with no finite element information know anything about degrees of freedom?

* What are "local" and "global" dofs in dofmap. Sometimes local seems to mean "local to a process", sometimes "local to a cell".

* Is a sub dofmap or an ad-hoc dofmap the right tool to implement DirichletBCs for Hermite elements? Rationale: `DirichletBC::compute_bc_pointwise|topological|whatever()` uses

* What is the best way of implementing `evaluate_dof()` for derivative evaluations? Do I need to extend the interface of GenericFunctions|whatever to include derivatives? Can I use symbolic / automatic differentiation?

In [ ]:
import mshr
domain = mshr.Rectangle(Point(0.0, 0.0), Point(2.0, 2.0))
mesh   = mshr.generate_mesh(domain, 10)